## 7324 Assignment A6 : NN and PCA
##### Name: Thang Nguyen
##### SMU ID: 48689334

## Imports

In [75]:
# The Classifiers
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.decomposition import PCA
# Data Wrangling Tool
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Foundational libraries
import pandas as pd
import tensorflow as tf
import keras
import numpy as np

## Loading Dataset

In [2]:
# note: I added the column names manually into the csv prior
# original csv from UCI comes without columns
abalone_df = pd.read_csv("../data/abalone.csv")
# checking dataframe
abalone_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sex             4177 non-null   object 
 1   Length          4177 non-null   float64
 2   Diameter        4177 non-null   float64
 3   Height          4177 non-null   float64
 4   Whole Weight    4177 non-null   float64
 5   Shucked Weight  4177 non-null   float64
 6   Viscera Weight  4177 non-null   float64
 7   Shell Weight    4177 non-null   float64
 8   Rings           4177 non-null   int64  
dtypes: float64(7), int64(1), object(1)
memory usage: 293.8+ KB


In [3]:
abalone_df.head()

,Sex,Length,Diameter,Height,Whole Weight,Shucked Weight,Viscera Weight,Shell Weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


## Cleaning Dataset

### One-hot Encoding abalone sex

In [4]:
abalone_df = pd.concat([abalone_df, pd.get_dummies(abalone_df['Sex'], prefix = 'Sex')], axis = 1)
abalone_df.drop(['Sex'], axis = 1, inplace = True)

In [5]:
abalone_df.head()

,Length,Diameter,Height,Whole Weight,Shucked Weight,Viscera Weight,Shell Weight,Rings,Sex_F,Sex_I,Sex_M
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15,0,0,1
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,0,0,1
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9,1,0,0
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10,0,0,1
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7,0,1,0


### Categorizing Rings into groups where
#### 0-4 -> infant
#### 5-9 -> young
#### 10-14 -> teen
#### 15-20 -> young adult
#### 20-25 -> adult

In [6]:
def group_rings(rings):
    if rings <= 4:
        return 0
    if rings <= 9:
        return 1
    if rings <= 14:
        return 2
    if rings <= 20: 
        return 3
    if rings <= 25:
        return 4

In [7]:
abalone_df["Rings"] = abalone_df["Rings"].apply(group_rings)

In [8]:
abalone_df.head()

,Length,Diameter,Height,Whole Weight,Shucked Weight,Viscera Weight,Shell Weight,Rings,Sex_F,Sex_I,Sex_M
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,3.0,0,0,1
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,1.0,0,0,1
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,1.0,1,0,0
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,2.0,0,0,1
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,1.0,0,1,0


## Analysis

### Split and Scale

In [9]:
target = abalone_df["Rings"].values
features = abalone_df.drop('Rings', axis=1)

In [10]:
# scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, target, random_state = 0, train_size = 0.8)

In [12]:
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.int32)

## Building Keras NN

### 1 input/output Layer

#### inputs: 10 features
#### output layer: 5 target groups

In [13]:
model = Sequential()

In [14]:
model.add(Dense(3, input_dim = X_train.shape[1], activation='relu'))
model.add(Dense(5, activation='softmax'))

In [15]:
# must use categorical and spars
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [16]:
model.fit(X_train, y_train, epochs=500)

Epoch 1/500
105/105 [==============================] - 0s 393us/step - loss: 1.4235 - accuracy: 0.5025
Epoch 2/500
  1/105 [..............................] - ETA: 0s - loss: 1.3725 - accuracy: 0.6250

2023-04-15 09:52:58.514173: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


105/105 [==============================] - 0s 331us/step - loss: 1.2357 - accuracy: 0.6315
Epoch 3/500
105/105 [==============================] - 0s 325us/step - loss: 1.1269 - accuracy: 0.6438
Epoch 4/500
105/105 [==============================] - 0s 449us/step - loss: 1.0569 - accuracy: 0.6468
Epoch 5/500
105/105 [==============================] - 0s 344us/step - loss: 1.0099 - accuracy: 0.6459
Epoch 6/500
105/105 [==============================] - 0s 336us/step - loss: 0.9770 - accuracy: 0.6459
Epoch 7/500
105/105 [==============================] - 0s 344us/step - loss: 0.9522 - accuracy: 0.6474
Epoch 8/500
105/105 [==============================] - 0s 329us/step - loss: 0.9330 - accuracy: 0.6498
Epoch 9/500
105/105 [==============================] - 0s 332us/step - loss: 0.9179 - accuracy: 0.6522
Epoch 10/500
105/105 [==============================] - 0s 330us/step - loss: 0.9059 - accuracy: 0.6567
Epoch 11/500
105/105 [==============================] - 0s 335us/step - loss: 0.8956

### 2 inner Layer

In [17]:
model = Sequential()

In [18]:
model.add(Dense(3, input_dim = X_train.shape[1], activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(5, activation='softmax'))

In [19]:
# must use categorical and spars
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [20]:
model.fit(X_train, y_train, epochs=500)

Epoch 1/500
105/105 [==============================] - 0s 440us/step - loss: 1.4720 - accuracy: 0.4565
Epoch 2/500
105/105 [==============================] - 0s 376us/step - loss: 1.2190 - accuracy: 0.6016
Epoch 3/500
105/105 [==============================] - 0s 360us/step - loss: 1.0599 - accuracy: 0.6468
Epoch 4/500
105/105 [==============================] - 0s 379us/step - loss: 0.9773 - accuracy: 0.6492
Epoch 5/500
105/105 [==============================] - 0s 356us/step - loss: 0.9399 - accuracy: 0.6504
Epoch 6/500
105/105 [==============================] - 0s 359us/step - loss: 0.9201 - accuracy: 0.6504
Epoch 7/500
105/105 [==============================] - 0s 354us/step - loss: 0.9073 - accuracy: 0.6507
Epoch 8/500
105/105 [==============================] - 0s 355us/step - loss: 0.8980 - accuracy: 0.6519
Epoch 9/500
105/105 [==============================] - 0s 360us/step - loss: 0.8907 - accuracy: 0.6501
Epoch 10/500
105/105 [==============================] - 0s 326us/step - l

### 3 inner Layers

In [21]:
model = Sequential()

In [22]:
model.add(Dense(3, input_dim = X_train.shape[1], activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(5, activation='softmax'))

In [23]:
# must use categorical and spars
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [24]:
model.fit(X_train, y_train, epochs=500)

Epoch 1/500
105/105 [==============================] - 0s 461us/step - loss: 1.3799 - accuracy: 0.4636
Epoch 2/500
105/105 [==============================] - 0s 406us/step - loss: 1.1808 - accuracy: 0.5334
Epoch 3/500
105/105 [==============================] - 0s 402us/step - loss: 1.0315 - accuracy: 0.6399
Epoch 4/500
105/105 [==============================] - 0s 399us/step - loss: 0.9508 - accuracy: 0.6453
Epoch 5/500
105/105 [==============================] - 0s 393us/step - loss: 0.9216 - accuracy: 0.6456
Epoch 6/500
105/105 [==============================] - 0s 427us/step - loss: 0.9022 - accuracy: 0.6474
Epoch 7/500
105/105 [==============================] - 0s 397us/step - loss: 0.8898 - accuracy: 0.6456
Epoch 8/500
105/105 [==============================] - 0s 401us/step - loss: 0.8796 - accuracy: 0.6465
Epoch 9/500
105/105 [==============================] - 0s 413us/step - loss: 0.8719 - accuracy: 0.6480
Epoch 10/500
105/105 [==============================] - 0s 405us/step - l

### Tuning: Increasing neurons using prime numbers => 
#### since feature set is relatively small but contains many observations,
#### increasing neurons aids in improving learning rate given 500 epochs

In [62]:
model = Sequential()

In [63]:
model.add(Dense(19, input_dim = X_train.shape[1], activation='relu'))
model.add(Dense(11, activation='relu'))
model.add(Dense(7, activation='relu'))
model.add(Dense(5, activation='softmax'))

In [64]:
# must use categorical and spars
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [65]:
model.fit(X_train, y_train, epochs=500)

Epoch 1/500
105/105 [==============================] - 0s 560us/step - loss: 1.2760 - accuracy: 0.5043
Epoch 2/500
105/105 [==============================] - 0s 602us/step - loss: 0.9832 - accuracy: 0.6453
Epoch 3/500
105/105 [==============================] - 0s 621us/step - loss: 0.9139 - accuracy: 0.6483
Epoch 4/500
105/105 [==============================] - 0s 587us/step - loss: 0.8775 - accuracy: 0.6612
Epoch 5/500
105/105 [==============================] - 0s 705us/step - loss: 0.8506 - accuracy: 0.6687
Epoch 6/500
105/105 [==============================] - 0s 602us/step - loss: 0.8280 - accuracy: 0.6681
Epoch 7/500
105/105 [==============================] - 0s 590us/step - loss: 0.8043 - accuracy: 0.6803
Epoch 8/500
105/105 [==============================] - 0s 572us/step - loss: 0.7834 - accuracy: 0.6770
Epoch 9/500
105/105 [==============================] - 0s 567us/step - loss: 0.7624 - accuracy: 0.6824
Epoch 10/500
105/105 [==============================] - 0s 574us/step - l

### Tuning: Adjusting loss function with increased neurons =>
#### Nadam is Adam with Nesterov momentum [Keras API doc]

In [241]:
model = Sequential()

In [242]:
model.add(Dense(19, input_dim = X_train.shape[1], activation='relu'))
model.add(Dense(17, activation='relu'))
model.add(Dense(13, activation='relu'))
model.add(Dense(5, activation='softmax'))

In [243]:
# must use categorical and sparse
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.Nadam(learning_rate=0.01),
              metrics=['accuracy'])

In [244]:
model.fit(X_train, y_train, epochs=500)

Epoch 1/500
105/105 [==============================] - 0s 609us/step - loss: 0.8588 - accuracy: 0.6327
Epoch 2/500
105/105 [==============================] - 0s 499us/step - loss: 0.7242 - accuracy: 0.6740
Epoch 3/500
105/105 [==============================] - 0s 491us/step - loss: 0.7044 - accuracy: 0.6899
Epoch 4/500
105/105 [==============================] - 0s 476us/step - loss: 0.6984 - accuracy: 0.6914
Epoch 5/500
105/105 [==============================] - 0s 472us/step - loss: 0.6991 - accuracy: 0.6959
Epoch 6/500
105/105 [==============================] - 0s 467us/step - loss: 0.6944 - accuracy: 0.6995
Epoch 7/500
105/105 [==============================] - 0s 478us/step - loss: 0.6831 - accuracy: 0.7043
Epoch 8/500
105/105 [==============================] - 0s 451us/step - loss: 0.6827 - accuracy: 0.7007
Epoch 9/500
105/105 [==============================] - 0s 466us/step - loss: 0.6799 - accuracy: 0.7004
Epoch 10/500
105/105 [==============================] - 0s 532us/step - l

### Tuning: Increasing Epochs with Nadam loss function and increased neurons =>
#### more epochs generally allows for more matured weights but learning can stagnate

In [249]:
model = Sequential()

In [250]:
model.add(Dense(19, input_dim = X_train.shape[1], activation='relu'))
model.add(Dense(17, activation='relu'))
model.add(Dense(13, activation='relu'))
model.add(Dense(5, activation='softmax'))

In [251]:
# must use categorical and sparse
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.Nadam(learning_rate=0.01),
              metrics=['accuracy'])

In [252]:
model.fit(X_train, y_train, epochs=1000)

Epoch 1/1500
105/105 [==============================] - 0s 575us/step - loss: 0.8921 - accuracy: 0.6318
Epoch 2/1500
105/105 [==============================] - 0s 454us/step - loss: 0.7342 - accuracy: 0.6779
Epoch 3/1500
105/105 [==============================] - 0s 445us/step - loss: 0.7137 - accuracy: 0.6914
Epoch 4/1500
105/105 [==============================] - 0s 505us/step - loss: 0.7050 - accuracy: 0.6956
Epoch 5/1500
105/105 [==============================] - 0s 473us/step - loss: 0.6951 - accuracy: 0.7037
Epoch 6/1500
105/105 [==============================] - 0s 460us/step - loss: 0.6911 - accuracy: 0.6920
Epoch 7/1500
105/105 [==============================] - 0s 464us/step - loss: 0.6831 - accuracy: 0.6980
Epoch 8/1500
105/105 [==============================] - 0s 453us/step - loss: 0.6799 - accuracy: 0.6956
Epoch 9/1500
105/105 [==============================] - 0s 458us/step - loss: 0.6788 - accuracy: 0.7088
Epoch 10/1500
105/105 [==============================] - 0s 460u